<a href="https://colab.research.google.com/github/vipashaaV321/User-Intent-Modeling/blob/main/NMF_ProfileLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from zipfile import ZipFile
from urllib.request import urlretrieve
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score
from sklearn.decomposition import NMF

In [ ]:
## extracting data
urlretrieve("http://files.grouplens.org/datasets/movielens/ml-1m.zip", "movielens.zip")
ZipFile("movielens.zip", "r").extractall()

In [ ]:
ratings = pd.read_csv(
    "ml-1m/ratings.dat",
    sep="::",
    names=["userId", "movieId", "rating", "unix_timestamp"],
)
ratings.head()
ratings_data = ratings

<ipython-input-3-bc50017c92e4>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv(


In [ ]:
# Correctly retrieve the number of unique users and movies
num_users = ratings_data['userId'].nunique()
num_movies = ratings_data['movieId'].nunique()

In [ ]:
# Find the maximum user and movie IDs
max_user_id = ratings_data['userId'].max()
max_movie_id = ratings_data['movieId'].max()

In [ ]:

# Create an interaction matrix of the appropriate size
interaction_matrix = np.zeros((max_user_id, max_movie_id))

In [ ]:
# Fill the interaction matrix with ratings
for _, row in ratings_data.iterrows():
    user_id = row['userId']
    movie_id = row['movieId']
    rating = row['rating']
    interaction_matrix[user_id - 1, movie_id - 1] = rating


In [ ]:
# Define the number of components for NMF
embedding_size = 8

In [ ]:
# Perform NMF on the interaction matrix
nmf_model = NMF(n_components=embedding_size)
user_embedding = nmf_model.fit_transform(interaction_matrix)
movie_embedding = nmf_model.components

In [ ]:
# Initialize user and movie selection
selected_user_id = 1
selected_movie_id = 1193

In [ ]:
while True:
    # Get recommendations based on dot product of user and movie embeddings
    user_profile = user_embedding[selected_user_id - 1, :]
    movie_scores = np.dot(user_profile, movie_embedding)
    top_movie_indices = np.argsort(movie_scores)[::-1]

    # Extract the top recommendations
    recommendations = top_movie_indices[:5]  # Show the top 5 recommendations

    print("Top 5 Recommendations:")
    for i, movie_index in enumerate(recommendations):
        movie_id = movie_index + 1
        print(f"{i + 1}. Movie ID: {movie_id}")

    # Ask the user to select a movie
    user_input = input("Select a movie by ID or enter 'end' to stop: ")

    if user_input == 'end':
      break

    # if user_input in recommendations:
    selected_movie_id = int(user_input)

    # Take user feedback after movie selection
    feedback_score = float(input("Enter feedback score (e.g., 1.0 to 5.0): "))  # User feedback
    user_embedding[selected_user_id - 1, :] +=  feedback_score * movie_embedding[:, selected_movie_id - 1]